In [23]:
import pandas as pd
from tqdm import tqdm_notebook
import numpy
from sklearn.model_selection import cross_val_score
from sklearn.utils import shuffle
import numpy as np
from sklearn.preprocessing import MinMaxScaler

import matplotlib.pyplot as plt 

from sklearn.ensemble import RandomForestClassifier
from sklearn.naive_bayes import GaussianNB
from sklearn.gaussian_process import GaussianProcessClassifier
from sklearn.svm import SVC
from sklearn.neighbors import KNeighborsClassifier
from sklearn.neural_network import MLPClassifier
from sklearn.tree import DecisionTreeClassifier
from sklearn.linear_model import LogisticRegression

In [27]:
%matplotlib inline

In [2]:
course_structure = pd.read_csv("kaggle/structure.csv")
activity = pd.read_csv("kaggle/user_activity.csv")

In [3]:
targets = pd.read_csv("kaggle/targets.csv")
test_activity = pd.read_csv("kaggle/user_activity_test.csv")
ones_only = pd.read_csv("kaggle/ones_only.csv")

In [4]:
course_structure

,course_id,module_id,module_position,lesson_id,lesson_position,step_id,step_position,step_type,step_cost
0,7,17,1,2721,1,7225,1,text,0
1,7,17,1,2721,1,7230,2,text,0
2,7,17,1,2721,1,7231,3,text,0
3,7,17,1,2721,1,7232,4,text,0
4,7,17,1,2721,1,43604,5,text,0
5,7,17,1,2721,1,7233,6,text,0
6,7,17,1,2721,1,10828,7,text,0
7,7,17,1,531,2,2828,1,text,0
8,7,17,1,531,2,4848,2,video,0
9,7,17,1,531,2,4850,3,video,0


In [4]:
activity

,user_id,action,step_id,time,step_type,step_cost
0,5920,discovered,2681,1403165936,text,0
1,5920,passed,2681,1403165936,text,0
2,5920,viewed,2681,1403165936,text,0
3,5920,passed,2682,1403165946,text,0
4,5920,viewed,2682,1403165946,text,0
5,5920,discovered,2682,1403165946,text,0
6,5920,passed,2687,1403165994,text,0
7,5920,discovered,2687,1403165994,text,0
8,5920,viewed,2687,1403165994,text,0
9,5920,passed,2688,1403166007,text,0


In [5]:
users_ids = sorted(activity.user_id.unique())

In [6]:
train_data = pd.DataFrame(index=users_ids)

# Attempt 2

In [8]:
order = course_structure.sort_values(['module_position', 'lesson_position', "step_position"])

In [14]:
order = np.array(order.step_id)

In [15]:
reverse_order = {}
count = 0
for step in order:
    reverse_order[step] = count
    count += 1

In [10]:
tasks = course_structure[course_structure.step_cost != 0].step_id

In [11]:
videos = course_structure[course_structure.step_type == "video"].step_id

In [12]:
users_actions = {}
for id in users_ids:
    users_actions[id] = activity[activity.user_id == id]

In [19]:
course_completion = {}
for id in tqdm_notebook(users_ids):
    completion = []
    for step in order:
        actions = users_actions[id][users_actions[id].step_id == step].action.unique()
        if "passed" in actions:
            completion.append(2)
        elif "viewed" in actions:
            completion.append(1)
        else:
            completion.append(0)
    course_completion[id] = completion

In [22]:
succes_rate = {}
for id in tqdm_notebook(users_ids):
        actions = users_actions[id][users_actions[id].step_cost != 0].action.value_counts().to_dict()
        if "passed" in actions.keys():
            succes_rate[id] = actions["passed"] / actions["started_attempt"]
        else:
            succes_rate[id] = 0

In [32]:
last_actions = {}
for id in tqdm_notebook(users_ids):
    count = 0
    last_step = 0
    for action in course_completion[id]:
        if action != 0:
            last_step = order[count]
        count += 1
    last_actions[id] = last_step

ERROR! Session/line number was not unique in database. History logging moved to new session 57



In [34]:
prices = {}
for action in tasks:
    prices[action] = int(course_structure[course_structure.step_id == action].step_cost) 

In [35]:
watched_videos_ration = {}
score_ratio = {}

for id in tqdm_notebook(users_ids):
    ind = 0
    score = 0
    overall_score = 0
    watched_videos = 0
    num_videos = 0
    while order[ind] != last_actions[id]:
        action_id = order[ind]
        if action_id in tasks.values:
            score += prices[action_id]
            if course_completion[id][ind] == 2:
                overall_score += prices[action_id]
        elif action_id in videos.values:
            num_videos += 1
            if course_completion[id][ind] == 2:
                watched_videos += 1
        ind += 1
        if ind >= len(order):
            break
            
    if num_videos == 0:
        watched_videos_ration[id] = 0
    else:
        watched_videos_ration[id] = watched_videos / num_videos
    if score == 0:
        score_ratio[id] = 0
    else:
        score_ratio[id] = overall_score / score

In [36]:
users_gaps = {}
for id in tqdm_notebook(users_ids):
    times = users_actions[id].time.unique()
    gaps = []
    for index in range(len(times) - 1):
        gaps.append(times[index+1] - times[index])
    users_gaps[id] = np.array(gaps).mean()
users_gaps = pd.Series(users_gaps).fillna(-1).to_dict()

/home/mdm/anaconda3/lib/python3.5/site-packages/numpy/core/_methods.py:59: RuntimeWarning: Mean of empty slice.
  warnings.warn("Mean of empty slice.", RuntimeWarning)


In [37]:
train_data['score_ratio'] = pd.Series(score_ratio)
train_data['watched_videos_ration'] = pd.Series(watched_videos_ration)
train_data['succes_rate'] = pd.Series(succes_rate)
train_data['users_gaps'] = pd.Series(users_gaps)

In [43]:
temp_y = pd.Series(targets['passed'], index=targets['user_id'])

In [44]:
train_data["target"] = temp_y

In [46]:
train_data.dropna(inplace=True)

## Beautiful function

In [57]:
def transform(activity, targets=[]):
    users_ids = sorted(activity.user_id.unique())
    
    train_data = pd.DataFrame(index=users_ids)
    
    order = course_structure.sort_values(['module_position', 'lesson_position', "step_position"])
    
    order = np.array(order.step_id)
    
    reverse_order = {}
    count = 0
    for step in order:
        reverse_order[step] = count
        count += 1

    tasks = course_structure[course_structure.step_cost != 0].step_id

    videos = course_structure[course_structure.step_type == "video"].step_id

    users_actions = {}
    for id in users_ids:
        users_actions[id] = activity[activity.user_id == id]

    course_completion = {}
    for id in tqdm_notebook(users_ids):
        completion = []
        for step in order:
            actions = users_actions[id][users_actions[id].step_id == step].action.unique()
            if "passed" in actions:
                completion.append(2)
            elif "viewed" in actions:
                completion.append(1)
            else:
                completion.append(0)
        course_completion[id] = completion

    succes_rate = {}
    for id in tqdm_notebook(users_ids):
        actions = users_actions[id][users_actions[id].step_cost != 0].action.value_counts().to_dict()
        if "passed" in actions.keys():
            succes_rate[id] = actions["passed"] / actions["started_attempt"]
        else:
            succes_rate[id] = 0

    last_actions = {}
    for id in tqdm_notebook(users_ids):
        count = 0
        last_step = 0
        for action in course_completion[id]:
            if action != 0:
                last_step = order[count]
            count += 1
        last_actions[id] = last_step

    prices = {}
    for action in tasks:
        prices[action] = int(course_structure[course_structure.step_id == action].step_cost)

    watched_videos_ration = {}
    score_ratio = {}

    for id in tqdm_notebook(users_ids):
        ind = 0
        score = 0
        overall_score = 0
        watched_videos = 0
        num_videos = 0
        while order[ind] != last_actions[id]:
            action_id = order[ind]
            if action_id in tasks.values:
                score += prices[action_id]
                if course_completion[id][ind] == 2:
                    overall_score += prices[action_id]
            elif action_id in videos.values:
                num_videos += 1
                if course_completion[id][ind] == 2:
                    watched_videos += 1
            ind += 1
            if ind >= len(order):
                break

        if num_videos == 0:
            watched_videos_ration[id] = 0
        else:
            watched_videos_ration[id] = watched_videos / num_videos
        if score == 0:
            score_ratio[id] = 0
        else:
            score_ratio[id] = overall_score / score

    users_gaps = {}
    for id in tqdm_notebook(users_ids):
        times = users_actions[id].time.unique()
        gaps = []
        for index in range(len(times) - 1):
            gaps.append(times[index+1] - times[index])
        users_gaps[id] = np.array(gaps).mean()
    users_gaps = pd.Series(users_gaps).fillna(-1).to_dict()

    train_data['score_ratio'] = pd.Series(score_ratio)
    train_data['watched_videos_ration'] = pd.Series(watched_videos_ration)
    train_data['succes_rate'] = pd.Series(succes_rate)
    train_data['users_gaps'] = pd.Series(users_gaps)

    if len(targets) != 0:
        temp_y = pd.Series(targets['passed'], index=targets['user_id'])

        train_data["target"] = temp_y

        train_data.dropna(inplace=True)
        
        x = train_data.drop("target", axis=1)
        y = train_data["target"]
        
        return x, y
    
    return train_data

## Check

In [64]:
x = train_data.drop("target", axis=1)
y = train_data["target"]

In [60]:
svc = SVC(class_weight="balanced")
lg = LogisticRegression(class_weight="balanced")
dtc = DecisionTreeClassifier(class_weight="balanced")

In [69]:
cross_val_score(dtc, x, y).mean()

0.75289965482250132

In [70]:
dtc.fit(x, y)

DecisionTreeClassifier(class_weight='balanced', criterion='gini',
            max_depth=None, max_features=None, max_leaf_nodes=None,
            min_impurity_split=1e-07, min_samples_leaf=1,
            min_samples_split=2, min_weight_fraction_leaf=0.0,
            presort=False, random_state=None, splitter='best')

## Predict

In [63]:
test_x =  transform(test_activity)

/home/mdm/anaconda3/lib/python3.5/site-packages/numpy/core/_methods.py:59: RuntimeWarning: Mean of empty slice.
  warnings.warn("Mean of empty slice.", RuntimeWarning)


In [71]:
predictions = pd.Series(dtc.predict(test_x), index=test_x.index)

In [72]:
answer = pd.DataFrame()
answer['user_id'] = predictions.index
answer['passed'] = predictions.values

In [73]:
answer['passed'] = answer['passed'].astype(int)

In [75]:
file = answer.to_csv(index=False)

In [76]:
f = open("answer.csv", "w")

In [77]:
f.write(file)

31842